In [ ]:
import getpass
import json

### Funciones auxiliares

In [20]:
def pedir_numero(mensaje):
    """Solicita y valida un número entero por consola.

    Parámetros:
    mensaje (str): Texto mostrado al pedir la entrada.
    Retorna:
    int: Número entero ingresado por el usuario.
    
    """
    while True:
        try:
            return int(input(mensaje))
        except ValueError:
            print("Esto no es un numero. ")

In [21]:
def mostrar_historial(lista):
    """Imprime el historial de operaciones almacenado.

    Parámetros:
        lista (list[str]): Lista con cadenas representando cada operación.

    """
    if len(lista) == 0:
        print("\nNo hay historial de operaciones por mostrar.")
    else:
        print("====Historial de operaciones====")
        for i, operacion in enumerate(lista, 1):
            print(f"{i}. {operacion}")

In [22]:
def repetir():
    """Pregunta al usuario si desea volver al menú principal.

    Retorna:
        bool: True si el usuario responde afirmativamente.
              False si responde negativamente.

    """
    while True:
        respuesta = input("\n¿Deseas volver al menu principal? (si/no): ").lower()
        if respuesta in ["si", "s"]:
            return True
        elif respuesta in ["no", "n"]:
            return False
        else:
            print("por favor ingresa 'si' o 'no'.")

### Funciones de operacion

In [23]:
def operacion(opcion, a, b):
    """Realiza la operación aritmética indicada e imprime el resultado.

    Parámetros:
        opcion (int): Código de operación
        a y b (int | float): numeros involucrados en la operación.

    """
    operaciones = {
        1 : ("Suma", a + b),
        2 : ("Resta", a - b),
        3 : ("Multiplicacion", a * b),
        4 : ("Division", None if b == 0 else a / b)
    }
    nombre, resultado = operaciones[opcion]
    if resultado is None:
        print("No se puede dividir entre 0.")
    else:
        print(f"\nLa {nombre} de {a} y {b} es: {resultado}")

In [24]:
def historial(lista, opcion, a, b):
    ''' 
    Parametros:
        lista: lista a modificar
        opcion: operacion a realizar
        a y b: numeros involucrados en la operacion '''
    
    operacion = {
        1 : ("+", (a + b)),
        2 : ("-", (a - b)),
        3 : ("*", (a * b)),
        4 : ("/", (a / b) if b != 0 else "Error")
    }        
    operador, resultado = operacion[opcion]
    entry = f"{a} {operador} {b} = {resultado}"
    lista.append(entry)

### Funciones de autenticacion

In [25]:
def cargar_datos():
    """Carga los datos de usuarios desde un archivo JSON.

    Retorna:
        dict: Diccionario con los datos de usuarios.

    """
    try:
        with open("usuarios.json", "r") as archivo:
            return json.load(archivo)
    except FileNotFoundError:
        return {}

In [26]:
def guardar_datos(diccionario):
    """Guarda los datos de usuarios en un archivo JSON.

    Parámetros:
        diccionario: Diccionario con los datos de usuarios.

    """
    with open("usuarios.json", "w") as archivo:
        json.dump(diccionario, archivo, indent=4)

In [27]:
def registro(diccionario):
    """Registra un nuevo usuario en el diccionario de usuarios.

    Parámetros:
        diccionario: Mapa usuario -> contraseña que se actualizara.

    """
    while True:
        usuario = input("Ingresa tu nombre de usuario: ")
        # Verificamos que el usuario no exista
        if usuario in diccionario:
            print("Este usuario ya existe en la base de datos.")
            continue
        else:
            password = getpass.getpass("Ingresa tu contrasena: ")
            diccionario[usuario] = {
                "password" : password,
                "historial" : []
            }
            guardar_datos(diccionario)
            print("Usuario registrado exitosamente.\n")
            break

In [28]:
def login(diccionario):
    """Autentica a un usuario contra el diccionario de usuarios.

    Parámetros:
        diccionario: Mapa usuario -> contraseña.
    Retorna:
        str | None: Nombre de usuario si la autenticacion es exitosa; None si falla.

    """
    while True:
        usuario = input("Ingresa tu nombre de usuario (o 'cancelar' para volver al menu): ")
        # Verificamos que el usuario no exista        
        if usuario.lower() == "cancelar":
            return None
        
        if usuario not in diccionario:
            print("Este usuario no existe en la base de datos.")
            continue
        else:
            for intento in range(3):
                password = getpass.getpass("Ingresa tu contraseña: ")
                if password == diccionario[usuario]["password"]:
                    print("¡login exitoso!")
                    return usuario
                else:
                    if intento < 2:
                        print(f"Contraseña invalida. Te quedan {2 - intento} intentos")
                    else:
                        print("Contraseña invalida.")
            print("Haz agotado tus intentos de contraseña")
            return None

In [29]:
def cambiar_password(diccionario, usuario):
    while True:
        actual = getpass.getpass("Contraseña actual: ")
        if actual == diccionario[usuario]["password"]:
            nueva1 = getpass.getpass("Nueva contraseña: ")
            nueva2 = getpass.getpass("Confirma nueva contraseña: ")
            if nueva1 == nueva2:
                diccionario[usuario]["password"] = nueva1
                guardar_datos(diccionario)
                print("Contraseña actualizada")
                break
            else:
                print("Las contraseñas no coinciden")
                continue
        else:
            print("Contraseña incorrecta")
            continue
    


In [30]:
def eliminar_cuenta(diccionario, usuario):
    print("\nEstas apunto de eliminar la cuenta")
    while True:
        try:
            respuesta = input("Deseas eliminar tu cuenta (si/no): ").strip().lower()
            if respuesta in ["si","s"]:
                del diccionario[usuario]
                guardar_datos(diccionario)
                print("\nTu cuenta ha sido eliminada.")
                return True
            else:
                if respuesta in ["no", "n"]:
                    return False
        except ValueError:
            print("Por favor, ingresa si o no")

    

### Funciones de Menu

In [31]:
def menu_acceso():
    """Muestra el menu de acceso (registro/login/salir) y valida la seleccion.

    Retorna:
        int: Opcion seleccionada.

    """
    print("\n=== Bienvenido ===")
    print("1. Registrarse")
    print("2. Iniciar sesión")
    print("3. Salir")
    while True:
        try:
            opcion_acceso = int(input("Que opcion quieres ejecutar: "))
            if opcion_acceso in [1, 2, 3]:
                print(f"\nHas seleccionado la opcion: {opcion_acceso}\n")
                break
            else:
                print("Por favor selecciona un numero entre 1 y 3.")
        except ValueError:
            print("Entrada no valida, por favor ingresa un numero")
    return (opcion_acceso)

In [32]:
def menu_gestion():

    print("\n=== Gestion de Cuenta ===")
    print("1. Cambiar contraseña")
    print("2. Eliminar Cuenta")
    print("3. Volver al menu principal")
    while True:
        try:
            opcion_acceso = int(input("Que opcion quieres ejecutar: "))
            if opcion_acceso in [1, 2, 3]:
                print(f"\nHas seleccionado la opcion: {opcion_acceso}\n")
                break
            else:
                print("Por favor selecciona un numero entre 1 y 3.")
        except ValueError:
            print("Entrada no valida, por favor ingresa un numero")
    return (opcion_acceso)

In [33]:
def menu():
    """Muestra el menu de operaciones y valida la seleccion.

    Retorna:
        int: Opcion seleccionada.

    """
    print("\nOperaciones: ")
    print("1. Suma \n2. Resta \n3. Multiplicacion \n4. Division \n5. Historial \n6. Gestion de cuenta \n7. Salir ")
    while True:
        try:
            opcion = int(input("Que opcion quieres ejecutar: "))
            if opcion in [1, 2, 3, 4, 5, 6, 7]:
                print(f"\nHas seleccionado la opcion: {opcion}")
                break
            else:
                print("Por favor selecciona un numero entre 1 y 8.")
        except ValueError:
            print("Entrada no valida, por favor ingresa un numero")
    return (opcion)

### Funcion Principal

In [34]:
def main():
    """Controla el flujo principal de la aplicacion (registro/login y operaciones).

    """
    usuarios = cargar_datos()
    usuario_actual = None

    while True:
        while True:
            opcion_acceso = menu_acceso()
            if opcion_acceso == 1:
                registro(usuarios) #Registro
                usuario_actual = login(usuarios)
                if usuario_actual:
                    break
            elif opcion_acceso == 2: #Login
                usuario_actual = login(usuarios)
                if usuario_actual:
                    break
            else:
                print("¡Hasta Luego!")
                return

        print(f"\nBienvenido a la calculadora, {usuario_actual}")        
        while True:
            opcion = menu()
            if opcion == 5:
                hist = usuarios.get(usuario_actual, {}).get("historial", [])
                mostrar_historial(hist)
                input("\nPresiona Enter para volver al menu...")

            elif opcion == 6:
                opcion_gestion = menu_gestion()
                if opcion_gestion == 1:
                    cambiar_password(usuarios, usuario_actual)
                elif opcion_gestion == 2:
                    if eliminar_cuenta(usuarios, usuario_actual):
                        print("volviendo al menu Login...")
                        break
                elif opcion_gestion == 3:
                    continue

            elif opcion == 7:
                print("¡Hasta luego!")
                return
            else:
                var_1 = pedir_numero("Ingresa un numero: ")
                var_2 = pedir_numero("Ingresa un numero: ")
                operacion(opcion, var_1, var_2)
                historial(usuarios[usuario_actual]["historial"], opcion, var_1, var_2)
                guardar_datos(usuarios)
                if not repetir():
                    print("\nGracias por usar la calculadora. ¡Hasta luego!")
                    return

In [35]:
main()


=== Bienvenido ===
1. Registrarse
2. Iniciar sesión
3. Salir

Has seleccionado la opcion: 2

¡login exitoso!

Bienvenido a la calculadora, Aaron

Operaciones: 
1. Suma 
2. Resta 
3. Multiplicacion 
4. Division 
5. Historial 
6. Gestion de cuenta 
7. Salir 

Has seleccionado la opcion: 3

La Multiplicacion de 89 y 12 es: 1068

Gracias por usar la calculadora. ¡Hasta luego!
